# Maritime Graph Weighting and Pathfinding Pipeline

This notebook provides a comprehensive, end-to-end workflow for applying intelligent weighting to a maritime navigation graph and calculating optimal routes.

## Workflow Overview

This is **Step 3** in the four-step maritime routing pipeline:

1. **Base Graph Creation** (`graph_PostGIS_v2.ipynb`): Creates coarse-resolution graph (0.3 NM spacing)
2. **Fine/H3 Graph Creation** (`graph_fine_PostGIS_v2.ipynb`): Creates high-resolution graph (0.02-0.3 NM or hexagonal)
3. **Graph Weighting & Pathfinding** (THIS NOTEBOOK): Applies intelligent weights and calculates routes
4. Configuration & Orchestration: Use `maritime_graph_postgis_workflow.py` for full automation

## What This Notebook Does

This notebook implements a **three-tier weighting system** that combines:

1. **Conversion**: Converting an undirected graph to a directed one to support traffic-flow constraints.
2. **Enrichment**: Adding S-57 feature data (depth, clearance, orientation) to graph edges for smart routing.
3. **Weighting**: Applying three tiers of weights:
   - **Static weights**: Distance-based penalties/bonuses from navigational features (land, fairways, TSS lanes)
   - **Directional weights**: Traffic flow alignment penalties/rewards (follow one-way lanes, align with fairways)
   - **Dynamic weights**: Vessel-specific constraints (draft, height, environmental conditions)
4. **Pathfinding**: Calculating optimal routes on the fully weighted graph using A* algorithm.

The entire pipeline is optimized for large graphs by performing all intensive operations directly within the PostGIS database. The graph is only loaded into memory at the final pathfinding step.

## Prerequisites

This notebook requires:
1. **Directed Graph** (or will create from undirected): Pre-computed fine/H3 graph from Step 2
2. **ENC Data**: S-57 charts converted to PostGIS format
3. **Configuration Files**:
   - `maritime_workflow_config.yml`: Workflow settings (vessel parameters, weights configuration)
   - `graph_config.yml`: Graph layer definitions and weight settings
4. **Database Setup**: PostGIS database with populated ENC schema
5. **Environment**: `.env` file with database credentials

**Setup Instructions:** See `docs/SETUP.md` for converting S-57 data to PostGIS backend.

**Troubleshooting:** See `docs/TROUBLESHOOTING.md` for common issues and solutions.

## 1. Notebook Configuration

Adjust the parameters in this section to control the notebook's behavior. You can run the entire weighting pipeline or toggle individual steps to skip completed operations.

### Key Configuration Notes:
- **Graph Names**: Must match output from fine/H3 graph creation step (Step 2)
- **Workflow Steps**: Set to `False` to skip already-completed operations (useful for re-running portions)
- **Vessel Parameters**: Adjust draft/height to match your specific vessel (affects navigable areas and route costs)
- **Usage Bands**: Controls which ENC chart scales contribute to static weighting (Band 3-5 recommended for coastal navigation)

See `maritime_workflow_config.yml` for production-ready configuration with full parameter documentation.

In [ ]:
# --- Graph Configuration ---
graph_name_undirected = "fine_graph_opt_20" # Source (undirected) graph
graph_name_directed = "fine_graph_directed_pg_20" # Target for directed, weighted graph

# --- Workflow Control ---
# Set these to True or False to enable/disable steps.
# For a full run, set all to True. For a partial run, disable completed steps.
workflow_steps = {
    "run_conversion_to_directed": True, # Creates the new directed graph tables
    "run_enrichment": True,             # Adds S-57 feature data (ft_*) to edges. REQUIRED for all weighting.
    "run_static_weights": True,         # Applies weights from static layers (land, fairways, etc.)
    "run_directional_weights": True,    # Applies weights based on traffic flow (TSS, fairways)
    "run_dynamic_weights": True,        # Applies final vessel-specific weights (draft, height)
    "run_pathfinding": True             # Loads the final graph and calculates a route
}

# --- Vessel & Environment Parameters (for Dynamic Weights & Pathfinding) ---
vessel_params = {
 'draft': 7.5,           # meters
 'height': 30.0,         # meters (for overhead clearance)
 'safety_margin': 2.0,   # meters (for under-keel clearance)
 'vessel_type': 'cargo'
}

env_conditions = {
 'weather_factor': 1.2,      # 1.0=good, >1.0=poor
 'visibility_factor': 1.1,   # 1.0=good, >1.0=poor
 'time_of_day': 'day'        # 'day' or 'night'
}

# --- Pathfinding Ports ---
departure_port_name = "SF Pilot"
arrival_port_name = "San Francisco Arrival"

departure_coords = {"lon": -122.780, "lat": 37.006}
arrival_coords = {"lon": -122.400, "lat": 37.805}

## 2. Setup and Initialization

This section imports all necessary libraries and initializes the core classes for the weighting pipeline:

- **ENCDataFactory**: Provides unified interface for accessing S-57 ENC data from PostGIS
- **H3Graph & FineGraph**: Manage graph operations (conversion, loading, saving, export)
- **Weights**: Implements the three-tier weighting system (static, directional, dynamic)
- **Route**: Calculates optimal routes on weighted graphs
- **PortData**: Manages port definitions (World Port Index + custom ports)

The output directory is created here for saving routes and benchmarks. Database credentials are loaded from the `.env` file (secure, not hardcoded).

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import time
import geopandas as gpd
import pandas as pd
import plotly.express as px
from shapely.geometry import Point

# --- Setup Python Environment ---
# Add the project root to the Python path to enable module imports
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# --- Import Maritime Module Components ---
from src.nautical_graph_toolkit.core.graph import H3Graph, Weights, FineTuning
from src.nautical_graph_toolkit.core.s57_data import ENCDataFactory
from src.nautical_graph_toolkit.core.pathfinding_lite import Route
from src.nautical_graph_toolkit.utils.port_utils import PortData

# Load environment variables from .env file at the project root
# This loads database credentials and API tokens
load_dotenv(project_root / ".env")

# --- Define Output Directory ---
output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

# --- Database Connection Parameters ---
# PostGIS connection parameters loaded from environment variables
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

# --- Initialize Core Classes ---
# These classes handle the entire weighting and pathfinding pipeline:
#
# ENCDataFactory: Provides unified interface for accessing S-57 ENC data
#   - Handles database connections and layer queries
#   - Used by all other classes for data access
#
# H3Graph: Manages graph operations (conversion, loading, saving)
#   - Converts undirected graphs to directed
#   - Loads/saves graphs from/to PostGIS
#   - Exports graphs to GeoPackage format
#
# Weights: Implements the three-tier weighting system
#   - Static weights: Distance-based penalties/bonuses from features
#   - Directional weights: Traffic flow alignment penalties/rewards
#   - Dynamic weights: Vessel-specific constraints (draft, height)
#   - Combines all tiers into final adjusted_weight
#
# PortData: Manages port definitions (World Port Index + custom ports)
factory = ENCDataFactory(source=db_params, schema="us_enc_all")
graph = H3Graph(data_factory=factory, route_schema_name="routes", graph_schema_name="graph")
weights_manager = Weights(data_factory=factory)
port_manager = PortData()

# --- Performance Tracking ---
# Dictionary to store timing metrics for each pipeline step
performance_metrics = {}

print("Setup complete. Core classes initialized.")

# --- Determine ENC List for the entire workflow ---
# This is done once at the start for efficiency and consistency across all steps.
# The ENC list defines which charts are relevant for the graph area and will be
# used for enrichment and weight calculations.
print("\nDetermining relevant ENCs from the source graph boundary...")
try:
    # Use the original undirected graph to define the geographic scope
    # by creating a convex hull around all graph nodes
    nodes_df_undirected = gpd.read_postgis(
        f'SELECT geometry FROM graph."{graph_name_undirected}_nodes"', 
        factory.manager.engine, 
        geom_col='geometry'
    )
    graph_boundary = nodes_df_undirected.geometry.union_all().convex_hull
    enc_list = factory.get_encs_by_boundary(graph_boundary)
    print(f"Found {len(enc_list)} ENCs for this workflow.")
    if not enc_list:
        print("Warning: No ENCs found for the graph boundary. Subsequent steps may fail.")
except Exception as e:
    print(f"Could not determine ENC list from source graph '{graph_name_undirected}'. Error: {e}")
    enc_list = []  # Ensure enc_list exists to avoid errors


## 3. Convert to Directed Graph

**Why this step is needed:** The undirected fine/H3 graph treats edges bidirectionally. However, maritime traffic often has directional rules:
- One-way traffic lanes (Traffic Separation Schemes)
- Current/wind patterns that favor certain directions
- Fairway orientation preferences

Converting to a directed graph allows us to:
1. Assign different weights to forward and reverse directions
2. Model one-way traffic lanes and channels
3. Apply directional bonuses/penalties based on traffic flow
4. Prepare for sophisticated routing that considers real-world constraints

**How it works:** Each undirected edge (A-B) becomes two directed edges (A→B and B→A). Feature data is propagated to both directions during enrichment. The operation is performed entirely on the database side for maximum performance.

In [ ]:
if workflow_steps["run_conversion_to_directed"]:
    start_time = time.perf_counter()
    print(f"Converting '{graph_name_undirected}' to directed graph '{graph_name_directed}'...")
    graph.convert_to_directed_postgis(
        source_table_prefix=graph_name_undirected,
        target_table_prefix=graph_name_directed,
        edges_schema="graph",
        drop_existing=True  # Ensures a clean start for the pipeline
    )
    end_time = time.perf_counter()
    performance_metrics['Conversion to Directed'] = end_time - start_time
    print("\nConversion to directed graph complete.")
else:
    print("Skipping conversion to directed graph.")

## 4. Enrich Edges with S-57 Features

This is a **critical prerequisite** for all weighting steps. Enrichment performs spatial joins to extract navigational data from S-57 ENC layers and attach it to graph edges.

### What Gets Enriched

The enrichment process adds `ft_*` columns to edges for each relevant S-57 feature:
- **Depth data** (`ft_depth`, `ft_valsou`): Water depth from soundings (for draft clearance checks)
- **Orientation** (`ft_orient`, `ft_trafic`): Traffic flow direction from TSS/fairways (for directional weights)
- **Clearance** (`ft_verclr`): Vertical clearance from bridges/cables (for height checks)
- **Feature presence** (`ft_lndare`, `ft_fairwy`, `ft_tsslpt`, etc.): Which feature types affect each edge
- **Safety characteristics**: Sounding values, construction types, precautionary area flags

### Why This is Required

All downstream weighting steps depend on this enriched data:
- **Static weights** need feature layers (`ft_lndare`, `ft_fairwy`, etc.) to calculate distance-based penalties/bonuses
- **Directional weights** need `ft_orient` and `ft_trafic` to align routes with traffic flow
- **Dynamic weights** need `ft_depth` and `ft_verclr` to enforce vessel-specific constraints

### Performance Note

This step can take 10-30 minutes for large graphs but only needs to be run once. All feature data is stored permanently in the edge table for subsequent weight calculations and is reusable for any routing scenario.

### Data Propagation

When `is_directed=True`, features are propagated to reverse edges to ensure bidirectional data consistency. This means:
- Forward edge (A→B) gets features from feature geometries
- Reverse edge (B→A) gets the same features
- Directional properties can then be applied differently to each direction during directional weighting

In [ ]:
if workflow_steps["run_enrichment"]:
    start_time = time.perf_counter()
    try:
        # Get the feature layers to extract from the S57 classifier
        # This defines which S-57 object classes will be joined to edges
        # (e.g., seaare, lndare, fairwy, tsslpt, soundg, etc.)
        feature_layers_to_enrich = weights_manager.get_feature_layers_from_classifier()

        # Run the server-side enrichment process
        # This performs spatial joins entirely in PostGIS for maximum performance:
        # - For each feature layer, finds edges that intersect or are near features
        # - Extracts relevant attributes (depth, orientation, clearance, etc.)
        # - Adds ft_* columns to the edges table with the extracted data
        # - Propagates data to reverse edges when is_directed=True
        print(f"Enriching {len(enc_list)} ENCs across {len(feature_layers_to_enrich)} feature layers...")
        summary = weights_manager.enrich_edges_with_features_postgis(
            enc_names=enc_list,  # ENCs determined from graph boundary in setup
            schema_name="graph",
            graph_name=graph_name_directed,
            enc_schema="us_enc_all",
            feature_layers=feature_layers_to_enrich,
            is_directed=True,  # IMPORTANT: Ensures features are propagated to reverse edges
            include_sources=False,  # Don't store ENC source names (saves space)
            soundg_buffer_meters=30  # Buffer for sounding point queries (depth data)
        )
        end_time = time.perf_counter()
        performance_metrics['Edge Enrichment'] = end_time - start_time
        print("\nEnrichment complete. Summary:")
        print(summary)
    except Exception as e:
        print(f"An error occurred during enrichment: {e}")
        print("Please ensure the directed graph tables exist.")
else:
    print("Skipping edge enrichment.")

## 5. Apply Static Weights

This step applies **distance-based penalties and bonuses** from static S-57 features. The three-tier system categorizes features as:

### Three-Tier Weight Categories

1. **Blocking Weights** (`wt_static_blocking`): Absolute avoidance zones
   - Land areas (`lndare`): factor = 100
   - Underwater rocks (`uwtroc`): factor = 100
   - Shoreline constructions (`slcons`): factor = 90
   - Edges within these features become effectively impassable

2. **Penalty Weights** (`wt_static_penalty`): Areas to avoid when possible
   - Currently not used in this configuration
   - Can be configured for areas like anchorages, restricted zones

3. **Bonus Weights** (`wt_static_bonus`): Preferred routing areas
   - Fairways (`fairwy`): factor = 0.5 (50% cost reduction)
   - Traffic Separation Schemes (`tsslpt`): factor = 0.7
   - Dredged areas (`drgare`): factor = 0.9
   - Precautionary areas (`prcare`): factor = 0.9
   - Recommended tracks (`rectrc`, `dwrtcl`): factor = 0.5

### Distance-Based Degradation

Weights are applied using a **distance degradation model** defined in `graph_config.yml`:
- Features have an influence `buffer` (e.g., 500m for underwater rocks)
- Weight intensity decreases with distance from feature
- Edges far from features get neutral weight (1.0)
- Edges near/within features get the configured factor

This allows smooth transitions between safe and dangerous areas rather than hard boundaries.

In [ ]:
if workflow_steps["run_static_weights"]:
    start_time = time.perf_counter()
    print("Applying static weights...")
    
    # Load configuration to get static layer definitions
    # graph_config.yml defines which layers are blocking/penalty/bonus
    # and the distance degradation parameters for each
    config = weights_manager._load_config()

    # Apply static weights using server-side PostGIS operations
    # This creates/updates three columns: wt_static_blocking, wt_static_penalty, wt_static_bonus
    # Each edge gets weights based on its spatial relationship to features
    weights_manager.apply_static_weights_postgis(
        graph_name=graph_name_directed,
        enc_names=enc_list,  # Use the ENCs determined in setup
        schema_name="graph",
        enc_schema="us_enc_all",
        static_layers=config["weight_settings"]["static_layers"],
        usage_bands=[3, 4, 5]  # Focus on higher-detail bands for static features
                               # Band 3: Approach (1:90K), Band 4: Harbour (1:22K-45K)
                               # Band 5: Berthing (1:4K-12K)
    )
    end_time = time.perf_counter()
    performance_metrics['Static Weights'] = end_time - start_time
    print("Static weights applied successfully.")
else:
    print("Skipping static weight application.")

## 6. Apply Directional Weights

This step calculates **traffic flow alignment penalties and rewards** based on how well an edge aligns with the intended direction of maritime traffic features.

### How Directional Weights Work

1. **Uses enriched orientation data** (`ft_orient` and `ft_trafic` from enrichment step):
   - `ft_orient`: The intended traffic direction (0-360 degrees) from features like TSS lanes and fairways
   - `ft_trafic`: Traffic direction code (1=one-way, 2=two-way)

2. **Calculates angular difference** between:
   - Edge direction (from start node to end node)
   - Feature orientation (intended traffic flow)

3. **Applies penalties/rewards based on alignment**:
   - **Aligned** (0-22.5°): Weight = 0.7 (30% cost reduction for following traffic)
   - **Slightly off** (22.5-45°): Weight = 1.2 (20% penalty)
   - **Moderately off** (45-90°): Weight = 5.0 (substantial penalty)
   - **Opposite direction** (135-180°): Weight = 50.0 (strongly discouraged)

4. **Two-way traffic handling**:
   - For two-way lanes (`ft_trafic=2`), checks if reverse edge is well-aligned
   - If reverse edge alignment > 95%, allows travel in both directions
   - Prevents penalizing legitimate two-way routes

### Affected Layers

Directional weights are applied to:
- Traffic Separation Schemes (`tsslpt`): One-way shipping lanes
- Fairways (`fairwy`): Main navigation channels
- Recommended tracks (`rectrc`, `dwrtcl`): Preferred routes

This ensures routes follow established maritime traffic patterns and avoid wrong-way travel in one-way lanes.

In [ ]:
if workflow_steps["run_directional_weights"]:
    start_time = time.perf_counter()
    print("Applying directional weights...")
    
    # Load configuration for angle bands and layer settings
    # graph_config.yml defines:
    # - Angle bands with thresholds and weight factors
    # - Layers to apply directional weights to
    # - Two-way traffic detection parameters
    config = weights_manager._load_config()
    directional_config = config["weight_settings"]["directional_weights"]
    two_way_config = directional_config.get("two_way_traffic", {})

    # Calculate directional weights using server-side PostGIS operations
    # This creates/updates the wt_dir column based on:
    # 1. Edge geometry direction (azimuth from start to end node)
    # 2. Feature orientation from ft_orient (extracted during enrichment)
    # 3. Traffic direction from ft_trafic (1=one-way, 2=two-way)
    weights_manager.calculate_directional_weights_postgis(
        schema_name="graph",
        graph_name=graph_name_directed,
        apply_to_layers=directional_config.get("apply_to_layers"),  # tsslpt, fairwy, rectrc, dwrtcl
        angle_bands=directional_config.get("angle_bands"),  # Angle thresholds and weights
        two_way_enabled=two_way_config.get("enabled", True),  # Enable two-way detection
        reverse_check_threshold=two_way_config.get("reverse_check_threshold", 95)  # 95% alignment
    )
    end_time = time.perf_counter()
    performance_metrics['Directional Weights'] = end_time - start_time
    print("Directional weights applied successfully.")
else:
    print("Skipping directional weight application.")

## 7. Apply Dynamic (Vessel-Specific) Weights

This is the **final weighting step** that combines all previous weights with vessel-specific constraints to produce the `adjusted_weight` used for pathfinding.

### Three-Tier Integration

Dynamic weights integrate all three tiers from previous steps:

1. **Tier 1 - Blocking Factor**: 
   - Combines static blocking weights with vessel constraints
   - Checks water depth (`ft_depth`) vs vessel draft + safety margin
   - Checks vertical clearance (`ft_verclr`) vs vessel height
   - Shallow water or low bridges result in extremely high weights (effectively blocking)

2. **Tier 2 - Penalty Factor**:
   - Combines static penalties with environmental conditions
   - Applies weather and visibility factors (e.g., 1.2x for poor weather)
   - Time-of-day adjustments (night navigation penalties)
   - Results in moderate weight increases for less favorable conditions

3. **Tier 3 - Bonus Factor**:
   - Uses static bonus weights (fairways, TSS lanes, etc.)
   - Applies vessel type preferences (cargo ships prefer deep channels)
   - Results in weight reductions for preferred routes

### Final Weight Calculation

The `adjusted_weight` for each edge is calculated as:
```
adjusted_weight = base_distance × blocking_factor × penalty_factor × bonus_factor × directional_weight
```

Where:
- `base_distance`: Original edge length in nautical miles (stored in `weight` column)
- `blocking_factor`: From `wt_static_blocking` + depth/clearance checks
- `penalty_factor`: From `wt_static_penalty` + environmental conditions
- `bonus_factor`: From `wt_static_bonus` + vessel preferences
- `directional_weight`: From `wt_dir` (traffic flow alignment)

**Result**: Edges that are safe, aligned with traffic, in preferred areas, and suitable for the vessel get low weights (preferred). Edges that are dangerous, misaligned, or unsuitable get high weights (avoided).

In [ ]:
if workflow_steps["run_dynamic_weights"]:
    start_time = time.perf_counter()
    print("Calculating final dynamic weights...")
    
    # This step performs the final weight integration entirely in PostGIS:
    # 1. Reads all previously calculated weight columns (wt_static_*, wt_dir)
    # 2. Reads enriched feature attributes (ft_depth, ft_verclr, etc.)
    # 3. Applies vessel constraint checks (draft vs depth, height vs clearance)
    # 4. Applies environmental condition factors (weather, visibility, time)
    # 5. Combines all factors into the final adjusted_weight column
    # 6. Preserves original 'weight' column (distance) for reference
    weights_manager.calculate_dynamic_weights_postgis(
        graph_name=graph_name_directed,
        schema_name="graph",
        vessel_parameters=vessel_params,  # From configuration cell: draft, height, type
        environmental_conditions=env_conditions,  # From configuration: weather, visibility, time
    )
    end_time = time.perf_counter()
    performance_metrics['Dynamic Weights'] = end_time - start_time
    
    print("Dynamic weights calculated successfully.")
    print("\nIMPORTANT: The 'adjusted_weight' column now contains the final routing weights.")
    print("           Use weight_key='adjusted_weight' in pathfinding operations.")
else:
    print("Skipping dynamic weight calculation.")

## 8. Pathfinding and Analysis

With the graph fully weighted, this final step loads the graph into memory and calculates an optimal route between a departure and arrival point. The route is then saved to a file for visualization in a GIS application.


### 8.1. Load Weighted Graph

This step loads the final, fully weighted graph from PostGIS into an in-memory `networkx` object. This can be time-consuming for large graphs.

In [ ]:
if workflow_steps["run_pathfinding"]:
    start_time = time.perf_counter()
    print(f"--- Loading final weighted graph '{graph_name_directed}' from PostGIS... ---")
    try:
        G = graph.load_graph_from_postgis(graph_name_directed)
        end_time = time.perf_counter()
        performance_metrics['Graph Loading'] = end_time - start_time
        print(f"Graph loaded with {G.number_of_nodes():,} nodes and {G.number_of_edges():,} edges.")
        print(f"Loading took {performance_metrics['Graph Loading']:.2f} seconds.")
    except Exception as e:
        print(f"Failed to load graph: {e}")
        G = None # Ensure G is None on failure
else:
    print("Skipping pathfinding step, graph will not be loaded.")
    G = None

### 8.2. Calculate and Save Route

Using the loaded graph, this step calculates the optimal route between the specified departure and arrival points using the final `adjusted_weight`.

In [ ]:
if workflow_steps["run_pathfinding"] and G is not None:
    start_time = time.perf_counter()
    print("\n--- Starting Route Calculation ---")

    # Create or update the custom port locations
    port_manager.create_custom_port(port_name=departure_port_name, lon=departure_coords['lon'], lat=departure_coords['lat'], if_exists='update')
    port_manager.create_custom_port(port_name=arrival_port_name, lon=arrival_coords['lon'], lat=arrival_coords['lat'], if_exists='update')

    # Get port geometries
    departure_port = port_manager.get_port_by_name(departure_port_name)
    arrival_port = port_manager.get_port_by_name(arrival_port_name)

    if departure_port.empty or arrival_port.empty:
        print("Error: Could not find departure or arrival port.")
    else:
        # Initialize the routing engine with the loaded graph
        route_finder = Route(graph=G, data_manager=factory.manager)

        # Calculate the detailed route
        print(f"Calculating route from '{departure_port_name}' to '{arrival_port_name}'...")
        route_detail = route_finder.detailed_route(
            departure_point=departure_port.geometry,
            arrival_point=arrival_port.geometry,
            weight_key='adjusted_weight' # CRITICAL: Use the final calculated weight
        )

        # Save the route to a file for visualization
        output_path = output_dir / f"detailed_route_{vessel_params['draft']}m_draft.geojson"
        route_finder.save_detailed_route_to_file(
            route_detail,
            output_path=str(output_path)
        )

        end_time = time.perf_counter()
        performance_metrics['Route Calculation'] = end_time - start_time
        print(f"\nPathfinding complete. Route saved to: {output_path}")
        print(f"Route calculation took {performance_metrics['Route Calculation']:.2f} seconds.")

elif workflow_steps["run_pathfinding"] and G is None:
    print("Skipping route calculation because the graph failed to load.")
else:
    print("Skipping route calculation.")

## 9. (Optional) Export Weighted Graph

If you need to use the final weighted graph in another application (like QGIS), you can export it from PostGIS to a GeoPackage file. This is much more efficient than loading it into memory first.

In [ ]:
# Set this to True if you want to export the final graph
run_export = True

if run_export:
    output_gpkg_path = output_dir / f"{graph_name_directed}.gpkg"
    print(f"Exporting '{graph_name_directed}' from PostGIS to '{output_gpkg_path}'...")

    # Use the efficient ogr2ogr-based export method
    try:
        graph.export_postgis_to_gpkg(
            schema_name="graph",
            graph_name=graph_name_directed,
            output_path=str(output_gpkg_path)
        )
        print("Export complete.")
    except FileExistsError:
        print(f"Export failed: File already exists at {output_gpkg_path}. Please delete it and try again.")
    except Exception as e:
        print(f"An error occurred during export: {e}")
else:
    print("Skipping graph export.")


## 10. Workflow Summary and Next Steps

Congratulations! You've completed the weighting and pathfinding pipeline. Here's what was accomplished:

### What You've Created

1. **Directed Graph** (`h3_graph_directed_pg_6_11`): Undirected graph converted to support directional routing
2. **Enriched Edges**: All edges now have S-57 feature attributes (depth, orientation, clearance, etc.)
3. **Three-Tier Weights**: Edges have static, directional, and dynamic weights combined into `adjusted_weight`
4. **Optimal Route**: A route computed using the final weighted graph that balances:
   - Safe passage (avoiding land, shallow areas, overhead hazards)
   - Traffic compliance (following fairways, TSS lanes, recommended tracks)
   - Vessel constraints (draft, height, type-specific preferences)
   - Environmental factors (weather, visibility, time of day)

### Understanding the Weights

The final `adjusted_weight` on each edge combines:
```
adjusted_weight = base_distance × blocking_factor × penalty_factor × bonus_factor × directional_weight
```

Where:
- **base_distance**: Original edge length (nautical miles)
- **blocking_factor**: Absolute obstacles (land, shallow water) - high = impassable
- **penalty_factor**: Areas to avoid (environmental conditions, hazards)
- **bonus_factor**: Preferred areas (fairways, TSS lanes, dredged channels) - <1.0 = encouraged
- **directional_weight**: Traffic flow alignment (follow one-way lanes, align with fairways)

### Next Steps

**For Further Analysis:**
- Examine route segments in QGIS to understand routing decisions
- Compare routes with different vessel parameters (draft, height)
- Analyze weight distributions to identify bottleneck areas
- Validate against real-world maritime practices

**For Production Use:**
- Use `maritime_graph_postgis_workflow.py` for automated full pipeline (Steps 1-4)
- Configure `maritime_workflow_config.yml` with your specific parameters
- Integrate routes into navigation systems, ETA calculators, or fuel estimation tools
- Update weighting factors based on operational experience and feedback

**For Performance Optimization:**
- Review benchmark metrics (`benchmark_graph_weighted_directed.csv`) to identify slow steps
- Consider using optimized save methods for future large graphs
- Experiment with different vessel parameters to understand weight sensitivities
- Profile the weighting steps that take longest for your specific AOI

### Benchmark Results

See the performance summary above for timing data on each step. Key metrics:
- **Edge Enrichment**: Usually the longest step (10-30 min) but runs once
- **Directional Weights**: Depends on feature coverage (5-15 min)
- **Dynamic Weights**: Combines all factors (~5-10 min)
- **Graph Loading**: Only needed for in-memory pathfinding (~10 min for large graphs)
- **Route Calculation**: Fast once graph is loaded (<1 min)

In [ ]:
# --- Visualize Pipeline Performance Metrics ---
# Create an interactive bar chart showing time taken for each pipeline step.
# This helps identify bottlenecks in the weighting workflow.
if performance_metrics:
    # Convert the dictionary to a pandas DataFrame for easy plotting
    perf_df = pd.DataFrame(list(performance_metrics.items()), columns=['Step', 'Time (seconds)'])
    perf_df = perf_df.sort_values(by='Time (seconds)', ascending=False)

    # Create an interactive bar chart
    fig = px.bar(
        perf_df,
        x='Step',
        y='Time (seconds)',
        title='Weighted Directed Graph Pipeline Performance',
        text_auto='.2f',
        labels={'Step': 'Pipeline Step', 'Time (seconds)': 'Time Taken (seconds)'}
    )
    fig.update_traces(textposition='outside')
    fig.show()
else:
    print("No performance metrics were recorded. Run one or more workflow steps to generate the summary.")

# --- Export Performance Benchmarks to CSV ---
# Save detailed performance metrics to CSV for long-term tracking and analysis.
# This allows comparison across different weighting configurations and graph sizes.
if performance_metrics:
    from datetime import datetime
    
    # Get graph statistics if graph was loaded
    if G is not None:
        node_count = G.number_of_nodes()
        edge_count = G.number_of_edges()
    else:
        # If graph wasn't loaded, query PostGIS for counts
        try:
            nodes_count_query = f'SELECT COUNT(*) FROM graph."{graph_name_directed}_nodes"'
            edges_count_query = f'SELECT COUNT(*) FROM graph."{graph_name_directed}_edges"'
            node_count = pd.read_sql(nodes_count_query, factory.manager.engine).iloc[0, 0]
            edge_count = pd.read_sql(edges_count_query, factory.manager.engine).iloc[0, 0]
        except:
            node_count = 0
            edge_count = 0
    
    # Calculate normalized metrics (per 100K nodes)
    time_per_100k_nodes = {}
    if node_count > 0:
        for step, time_val in performance_metrics.items():
            time_per_100k_nodes[step] = (time_val / node_count) * 100000
    
    # Build benchmark record with metadata and metrics
    benchmark_record = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'workflow': 'graph_weighted_directed_postgis_v2',
        'data_source': 'PostGIS',
        'db_schema': 'graph',
        'graph_name': graph_name_directed,
        'node_count': node_count,
        'edge_count': edge_count,
        'vessel_draft_m': vessel_params['draft'],
        'vessel_height_m': vessel_params['height'],
        'vessel_type': vessel_params['vessel_type'],
        'weather_factor': env_conditions['weather_factor'],
        'enc_count': len(enc_list),
        # Individual timing metrics (in seconds)
        'conversion_to_directed_sec': performance_metrics.get('Conversion to Directed', 0),
        'edge_enrichment_sec': performance_metrics.get('Edge Enrichment', 0),
        'static_weights_sec': performance_metrics.get('Static Weights', 0),
        'directional_weights_sec': performance_metrics.get('Directional Weights', 0),
        'dynamic_weights_sec': performance_metrics.get('Dynamic Weights', 0),
        'graph_loading_sec': performance_metrics.get('Graph Loading', 0),
        'route_calculation_sec': performance_metrics.get('Route Calculation', 0),
        # Normalized metrics (time per 100K nodes)
        'conversion_per_100k_nodes': time_per_100k_nodes.get('Conversion to Directed', 0),
        'enrichment_per_100k_nodes': time_per_100k_nodes.get('Edge Enrichment', 0),
        'static_weights_per_100k_nodes': time_per_100k_nodes.get('Static Weights', 0),
        'directional_weights_per_100k_nodes': time_per_100k_nodes.get('Directional Weights', 0),
        'dynamic_weights_per_100k_nodes': time_per_100k_nodes.get('Dynamic Weights', 0),
        'graph_loading_per_100k_nodes': time_per_100k_nodes.get('Graph Loading', 0),
        'route_calculation_per_100k_nodes': time_per_100k_nodes.get('Route Calculation', 0),
        # Total pipeline time
        'total_pipeline_sec': sum(performance_metrics.values()),
    }
    
    # Convert to DataFrame
    benchmark_df = pd.DataFrame([benchmark_record])
    
    # Define CSV file path (separate file for weighted graph workflow)
    benchmark_csv = output_dir / 'benchmark_graph_weighted_directed.csv'
    
    # Append to existing CSV or create new one
    if benchmark_csv.exists():
        existing_df = pd.read_csv(benchmark_csv)
        combined_df = pd.concat([existing_df, benchmark_df], ignore_index=True)
        combined_df.to_csv(benchmark_csv, index=False)
        print(f"\nAppended benchmark to existing file: {benchmark_csv}")
        print(f"Total benchmark records: {len(combined_df)}")
    else:
        benchmark_df.to_csv(benchmark_csv, index=False)
        print(f"\nCreated new benchmark file: {benchmark_csv}")
    
    # Display the current benchmark record
    print("\n=== Current Benchmark Record ===")
    print(f"Timestamp: {benchmark_record['timestamp']}")
    print(f"Workflow: {benchmark_record['workflow']}")
    print(f"Graph: {benchmark_record['graph_name']}")
    print(f"Nodes: {benchmark_record['node_count']:,}")
    print(f"Edges: {benchmark_record['edge_count']:,}")
    print(f"Vessel: {benchmark_record['vessel_type']} (draft={benchmark_record['vessel_draft_m']}m, height={benchmark_record['vessel_height_m']}m)")
    print(f"ENCs processed: {benchmark_record['enc_count']}")
    print(f"Total Pipeline Time: {benchmark_record['total_pipeline_sec']:.2f}s")
    print(f"\nMost demanding operations:")
    
    # Show top 3 time-consuming operations
    top_operations = sorted(
        [(k, v) for k, v in performance_metrics.items()],
        key=lambda x: x[1],
        reverse=True
    )[:3]
    for i, (op, time_val) in enumerate(top_operations, 1):
        print(f"  {i}. {op}: {time_val:.2f}s")
else:
    print("No performance metrics to export.")